In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import os

data_directory = os.path.join("data", 'mimic_III')
target_csv = os.path.join(data_directory, 'admissions.csv')

# Load your dataset
df = pd.read_csv(target_csv)

# Drop identifying columns
df = df.drop(columns=["row_id", "subject_id", "hadm_id"], errors='ignore')

# Handle missing values
# df = df.fillna(df.median(numeric_only=True))



In [5]:
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from classes.synthetic_data_generation_model import PATEGANSynthesizer

PATE_PATH = os.path.join('models', 'pate_gan_model.pkl')

# Create metadata (auto-detected or manual)
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)

# Initialize PATE-GAN with privacy parameters
pate_gan = PATEGANSynthesizer(
    metadata=metadata,
    num_teachers=10,
    epsilon=0.1,
    latent_dim=128
)

# Fit with automatic preprocessing
pate_gan.fit(df)

# Generate with automatic postprocessing
synthetic_data = pate_gan._sample(1000)

# # Evaluate using SDV metrics    
# quality_report = evaluate_quality(
#     real_data=df,
#     synthetic_data=synthetic_data,
#     metadata=metadata
# )

# Save/load with full state preservation
pate_gan.save(PATE_PATH)
loaded_model = PATEGANSynthesizer.load(PATE_PATH)

c:\Users\Admin\Documents\Main\MedSync\project-validation-platform-mockup\pvp-mockup-be\.venv\Lib\site-packages\sdv\single_table\base.py:163: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
c:\Users\Admin\Documents\Main\MedSync\project-validation-platform-mockup\pvp-mockup-be\.venv\Lib\site-packages\sdv\single_table\base.py:129: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
c:\Users\Admin\Documents\Main\MedSync\project-validation-platform-mockup\pvp-mockup-be\.venv\Lib\site-packages\sdv\_utils.py:304: SDVVersionWarning: You are currently on SDV version 1.24.0 but this synthesizer was created on version None. The latest bug fixes and features may not be available for this synthesizer. To see these enhancements, create and train a new synthesizer on this version.
  warnings.warn(message, SDVVe

In [6]:
pate_gan._sample(100)

,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,2102-08-29 07:15:00,2102-09-06 16:20:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SNF,Medicare,ENGL,JEWISH,MARRIED,WHITE,2104-09-24 12:07:00,2104-09-24 18:50:00,PNEUMONIA,0,1
1,2102-08-29 07:15:00,2102-09-06 16:20:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,2104-09-24 12:07:00,2104-09-24 18:50:00,ACUTE CHOLANGITIS,0,1
2,2102-08-29 07:15:00,2102-09-06 16:20:00,2105-06-11 02:20:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,JEWISH,MARRIED,WHITE,2104-09-24 12:07:00,2104-09-24 18:50:00,PNEUMONIA,1,1
3,2102-08-29 07:15:00,2102-09-06 16:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,JEWISH,MARRIED,WHITE,NaN,2104-09-24 18:50:00,LEFT HIP OA/SDA,0,1
4,2102-08-29 07:15:00,2102-09-06 16:20:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SNF,Medicare,ENGL,UNOBTAINABLE,MARRIED,WHITE,2104-09-24 12:07:00,2104-09-24 18:50:00,PNEUMONIA,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2102-08-29 07:15:00,2102-09-06 16:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,PNEUMONIA,0,1
96,2102-08-29 07:15:00,2102-09-06 16:20:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SNF,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,2104-09-24 12:07:00,2104-09-24 18:50:00,STEMI;,0,1
97,2102-08-29 07:15:00,2102-09-06 16:20:00,2105-06-11 02:20:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SNF,Medicare,ENGL,UNOBTAINABLE,MARRIED,WHITE,2104-09-24 12:07:00,NaN,URINARY TRACT INFECTION;PYELONEPHRITIS,0,1
98,2102-08-29 07:15:00,2102-09-06 16:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,JEWISH,MARRIED,WHITE,NaN,2104-09-24 18:50:00,FEVER;URINARY TRACT INFECTION,1,1
